In [ ]:
from google.colab import drive
drive.mount('/gdrive')
import os

os.chdir('/gdrive/MyDrive/')


Mounted at /gdrive


In [ ]:
!pip install GPy
!pip install sklearn
!pip install tensorboardX

     |████████████████████████████████| 1.0MB 16.9MB/s 
     |████████████████████████████████| 71kB 9.6MB/s 
  Created wheel for GPy: filename=GPy-1.9.9-cp37-cp37m-linux_x86_64.whl size=2626981 sha256=e219fa70bec53676afed63b25d94b82f01c6a07c1ae4626a28e2ee98bd700ae1
  Stored in directory: /root/.cache/pip/wheels/5d/36/66/2b58860c84c9f2b51615da66bfd6feeddbc4e04d887ff96dfa
  Created wheel for paramz: filename=paramz-0.9.5-cp37-none-any.whl size=102552 sha256=4a8443986d48e1ab161a89eeba774a0e19f0601f43b53089725a3ce3d95707ea
  Stored in directory: /root/.cache/pip/wheels/c8/4a/0e/6e0dc85541825f991c431619e25b870d4b812c911214690cf8
Successfully built GPy paramz
     |████████████████████████████████| 122kB 15.9MB/s 


In [ ]:
#The 3 following cells aim at simuling the behavior of NN models with a simple model for MNIST
!pip install ray==1.1
!pip install -U hyperopt
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch.nn as nn
import torch
import torch.nn.functional as F
import math
import numpy as np
import random
import math 
from ray import tune
from hyperopt import hp, fmin, tpe, Trials
from functools import *
from ray.tune.logger import *
import time

EPOCH_SIZE = 32*32*32*32
TEST_SIZE = 256*32*32 #remove 1024

#This is a function that can be used by several NN model
def train(model, optimizer ,func ,train_loader):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.train()
    #for (data, target) in train_loader:
    for batch_idx, (data, target) in enumerate(train_loader):
        # We set this just for the example to run quickly.
        if batch_idx * len(data) > EPOCH_SIZE:
           # print("hehe")
            return
        # We set this just for the example to run quickly.
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = func(output, target)
        loss.backward()
        optimizer.step()
        
#This is a function that can be used by several NN model (it only does accuracy ATM)
def test(model, func, data_loader):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (data, target) in enumerate(data_loader):
            # We set this just for the example to run quickly.
            data, target = data.to(device), target.to(device)
            outputs = model(data)
            _, predicted = torch.max(outputs.data, 1)

            total += target.size(0)
            correct += (predicted == target).sum().item()


                
    return correct / total

     |████████████████████████████████| 48.5MB 62kB/s 
     |████████████████████████████████| 71kB 10.0MB/s 
     |████████████████████████████████| 3.1MB 45.3MB/s 
     |████████████████████████████████| 204kB 56.4MB/s 
     |████████████████████████████████| 133kB 44.4MB/s 
     |████████████████████████████████| 1.3MB 37.3MB/s 
     |████████████████████████████████| 81kB 12.3MB/s 
     |████████████████████████████████| 81kB 11.9MB/s 
     |████████████████████████████████| 92kB 12.8MB/s 
     |████████████████████████████████| 296kB 59.6MB/s 
     |████████████████████████████████| 143kB 58.6MB/s 
  Created wheel for gpustat: filename=gpustat-0.6.0-cp37-none-any.whl size=12621 sha256=49f1a416f77f0c0185284c24a670006c3740545dff86eca3d7be16ae47c2145d
  Stored in directory: /root/.cache/pip/wheels/48/b4/d5/fb5b7f1d040f2ff20687e3bad6867d63155dbde5a7c10f4293
Successfully built gpustat
     |████████████████████████████████| 972kB 19.9MB/s 
  Found existing installation: hyperopt 0.1.2


In [ ]:
# A random mnist from the internet to get a correct model to reason about

class train_mnist():
    def __init__(self,config):
        
        self.config = {
        "sigmoid_func": 1
      ,  "hidden_dim":43
      ,  "n_layer":2    }
        for key, value in config.items():
            self.config[key] = value
        config = self.config
        
        self.i = 0
        
       # mnist_transforms = transforms.Compose(
       #     [transforms.ToTensor(),
       #      transforms.Normalize((0.1307, ), (0.3081, ))])
        mnist_transforms = transforms.ToTensor()

        self.train_loader = DataLoader(
            datasets.MNIST("/gdrive/MyDrive", train=True, download=True , transform=mnist_transforms),
            batch_size=64,
            shuffle=True)
       # self.test_loader = DataLoader(
       #     datasets.MNIST("/gdrive/MyDrive", train=False, transform=mnist_transforms),
       #     batch_size=64,
       #     shuffle=True)

        
        test_valid_dataset = datasets.MNIST("/gdrive/MyDrive", train=False, transform=mnist_transforms)
        valid_ratio = 0.5  
        nb_test = int((1.0 - valid_ratio) * len(test_valid_dataset))
        nb_valid =  int(valid_ratio * len(test_valid_dataset))
        test_dataset, val_dataset = torch.utils.data.dataset.random_split(test_valid_dataset, [nb_test, nb_valid])
        self.test_loader =  DataLoader(test_dataset,
            batch_size=64,
            shuffle=True)

        self.val_loader =  DataLoader(val_dataset,
            batch_size=64,
            shuffle=True)

        sigmoid_func_uniq = nn.Tanh()


        self.model = LeNet(192,int(round(config.get("hidden_dim",64))),10,
                    int( round(config.get("n_layer",1))),
                    config.get("droupout_prob",0.5) ,sigmoid_func_uniq)
        
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr=config.get("lr", 0.01), 
                                     betas=((config.get("b1", 0.999),config.get("b2", 0.9999))),
                                     eps=config.get("eps", 1e-08), 
                                     weight_decay=config.get("weight_decay", 0), 
                                     amsgrad=True)
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.model.to(device)
    
    def adapt(self, config):
        #print(self.optimizer)
        temp = copy.deepcopy(self)
        for key, value in config.items():
            temp.config[key] = value
        config = temp.config

        temp.model.adapt(config.get("droupout_prob", 0.5))
        temp.optimizer = torch.optim.Adam(temp.model.parameters(), lr=config.get("lr", 0.01), 
                                     betas=((config.get("b1", 0.999),config.get("b2", 0.9999))),
                                     eps=config.get("eps", 1e-08), 
                                     weight_decay=config.get("weight_decay", 0), 
                                     amsgrad=True)
        return temp
    
    
# All NN models should have a function train1 and test1 that calls the common train and test defined above.
# train1 and test1 is then used in the scheduler
    def train1(self):
        print("iteration: " + str(self.i) )
        self.i+=1
        train(self.model, self.optimizer, F.nll_loss, self.train_loader)

    def val1(self):
        return test(self.model, F.nll_loss, self.val_loader)
        
    def test1(self):
        return test(self.model, F.nll_loss, self.test_loader)

    def step(self):
        train1()
        return val1()

# __INCEPTION_SCORE_begin__
class LeNet(nn.Module):
    """
    LeNet for MNist classification, used for inception_score
    """

    def __init__(self,input_dim, hidden_dim, output_dim, n_layers,
            drop_prob, sigmoid ):
        super(LeNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d(drop_prob)
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def adapt(self,drop_prob):
        self.conv2_drop = nn.Dropout2d(drop_prob)
        
    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)



# Convolution Neural network using Pytorch 
class ConvNet(nn.Module):
    def __init__(self,input_dim, hidden_dim, output_dim, n_layers,
                 drop_prob, sigmoid ):
        super(ConvNet, self).__init__()
        
        self.sigmoid = sigmoid
        self.i_d = input_dim
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        self.conv1 = nn.Conv2d(1, 3, kernel_size=3)

        self.fc = nn.Linear(input_dim, output_dim)
        self.first= nn.Linear(input_dim, hidden_dim)
        self.hidden = [nn.Linear(hidden_dim,hidden_dim) for _ in range(self.n_layers)]
        self.drop_out = nn.Dropout(drop_prob)

        self.last = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        x = self.sigmoid(F.max_pool2d(self.conv1(x), 3))
        x = x.view(-1, self.i_d)
        x=self.first(x)
        x=self.drop_out(x)
        for i in range(self.n_layers):
            x=self.hidden[i](x)
            x=self.drop_out(x)
        x = self.last(x)
        return F.log_softmax(x, dim=1)


NameError: name 'LinearReg' is not defined

In [ ]:
class Parent():
    """Parent Class that handles the passage of Network Configurations from one step to the
    following
    """
    def __init__(self, point_hyperspace, configuration, model, loss):
        self.point_hyperspace = point_hyperspace
        self.configuration_list = [configuration]
        self.loss_list = [np.array(loss)]
        self.model = model
        self.is_replicated = False

    def update(self, configuration, loss, model):
        self.is_replicated = False
        self.configuration_list.append(configuration)
        self.loss_list=np.append(self.loss_list,loss)
        self.model = model

    def replication(self, n_children):
        self.is_replicated = True
      #  self.configuration_list.append(self.configuration_list[-1])
      #  self.loss_list=np.append(self.loss_list,self.loss_list[-1])
    #    replication_trials(self.point_hyperspace.trials, n_children)

    def get_last_conf(self):
        return self.configuration_list[-1]

    def get_point_hyperspace(self):
        return self.point_hyperspace

    def get_model(self):
        return self.model

    def get_loss(self):
        return self.loss_list

0.3674


0.6875


In [9]:
import copy


class Scheduler():
    def __init__(self, model, num_iteration, num_config,
                 oracle):
        #Oracle manages the Bayesian optimization
        self.oracle = oracle
        self.iteration = num_iteration
        self.num_config = num_config 
        self.sqrt_config =   math.floor(math.sqrt(num_config)) # math.ceil(num_config/5) #
        self.n_parents = self.sqrt_config
        #self.h is for the m "h" used at every loop, h is a configuration from the search space
        self.h = np.repeat({},num_config) 
        
        #self.out is for storing the result of the algorithm, ie all "h" from all iterations
        #from all sqrt(m) best models per iterations.
        self.out = np.zeros((num_iteration,self.sqrt_config))
        
        #self.hyperspaces is for storing the sqrt(m) hyperspaces used by the algorithm
        self.hyperspaces = np.zeros(self.sqrt_config)
        
        self.plot = np.zeros(num_iteration)

        
        #self.model is the m model that will explore new hyperspace points at every iterations
        self.models = np.repeat(model,num_config)
        
        #self.parents is the sqrt(m) best model from last iteration
        self.parents = np.repeat(model,self.sqrt_config)

        #self.losses remembers the performances of all m models at one iteration to decide which ones are the sqrt(m) best from self.models.
        self.losses = np.zeros(num_config)
        
        self.k = [0] # c'est pour avoir un pointeur sur k, c'est pas plus que O(sqrt)-paralélisable  pour le moment du coup.
    
    def initialisation(self):
        num_config = self.num_config
        extended_Hyperspace = Trials()
        fmin_objective = partial(test_function, models=self.models,h=self.h,losses=self.losses,parent_model=self.models, k_f = self.k,iteration = 0)
        self.oracle.compute_batch(extended_Hyperspace ,num_config , 0 ,fmin_objective)
            
        indexes = np.argsort(self.losses)     
        self.out[0] = (self.losses[indexes])[0:self.sqrt_config]
        self.hyperspaces = np.repeat(extended_Hyperspace,self.sqrt_config)    
        self.parents = np.array([Parent(copy.deepcopy(extended_Hyperspace),(self.h[indexes])[i], (self.models[indexes])[i],(self.losses[indexes])[i])  
                                 for i in range(self.sqrt_config) ])         
        self.plot[0] = self.losses[indexes][0]
        
    def loop(self):
        sqrt_config = self.sqrt_config
        iteration = self.iteration
        for i in range(1,iteration):
            
            self.k[0] = 0
            
            start_time = time.time()
            for j in range(sqrt_config):
                parent = self.parents[j]
                point_extended_hyperspace = parent.get_point_hyperspace()
                print("\n loss of parent " + str(parent.get_loss()[-1]) )
                print("\n loss " + str(parent.get_loss()))
                
                fmin_objective = partial(test_function, models=self.models,h=self.h,losses=self.losses,parent_model=parent.get_model(), k_f = self.k,iteration = len(parent.get_loss()))

                    
                if not parent.is_replicated:
                   # point_extended_hyperspace = Trials()
                   # parent.point_hyperspace = Trials()
                    print('not replicated')
                    self.oracle.repeat_good(
                        point_extended_hyperspace,
                        len(parent.get_loss()),
                        fmin_objective,
                        parent.configuration_list[-1]
                    )

                    # computes the new batch for each one of the parents for every iteration
                    self.oracle.compute_batch(
                        point_extended_hyperspace,
                        int(self.num_config/self.n_parents) - 1,
                        len(parent.get_loss()),
                        fmin_objective
                    )
                else:

                    print('replicated')
                    self.oracle.compute_batch(
                        point_extended_hyperspace,
                        int(self.num_config/self.n_parents),
                        len(parent.get_loss()),
                        fmin_objective
                    )
                    

            #self.oracle.Repeat_good(extended_Hyperspace ,i ,fmin_objective,parent.configuration_list[-1])
             #   self.oracle.compute_Batch(extended_Hyperspace ,int(self.num_config/sqrt_config) -1 , i ,fmin_objective)
           
            print("totalt time: " +  str(time.time() - start_time))


            combined_losses = np.concatenate(
                        (
                            self.losses,
                            
                                [self.parents[i].get_loss()[-1].item() for i in range(self.n_parents)]
                            
                        ),
                        0
                    )
            ixs_parents = np.argsort(combined_losses)
            parent_idx = ixs_parents[:self.n_parents]
            print(combined_losses)
            print(parent_idx)
            # ??? why saving it in a numpt array ?
            # It is creating the new Parent `array`
            temp_parents = [''] * self.n_parents

            for j, x in enumerate(parent_idx):
                # ??? why converting it to integer ?
                x = int(x)
                if x >= self.num_config:
                    temp_parents[j] = copy.deepcopy(self.parents[x - self.num_config])
                    temp_parents[j].replication(self.n_parents)
                else:
                    temp_parents[j] = copy.deepcopy(self.parents[math.floor(x/self.num_config * self.n_parents)])
                    temp_parents[j].update(self.h[x], self.losses[x], self.models[x])
            self.parents = temp_parents


class FSVNLogger(tune.logger.Logger):
    def _init(self):
        progress_file = os.path.join(path, "trash.csv")
        self._continuing = os.path.exists(progress_file)
        self._file = open(progress_file, "a")
        self._csv_out = None
    def on_result(self, result):
        tmp = result.copy()
        result = flatten_dict(tmp, delimiter="/")
        if self._csv_out is None:
            self._csv_out = csv.DictWriter(self._file, result.keys())
            self._csv_out.writeheader()

            #if not self._continuing:
        self._csv_out.writerow(
            {k: v
             for k, v in result.items() if k in self._csv_out.fieldnames})
        self._file.flush()
                   

config= {
     "lr": hp.uniform("lr",0,.1)
    , "droupout_prob": hp.uniform("droupout_prob",0,1)
          ,   "weight_decay": hp.uniform("weight_decay",0,.1),
    "b1" : hp.uniform("b1",0.9, 1),
    "b2" : hp.uniform("b2",0.99, 1)
}
fsvnlogger = FSVNLogger(config,"")

    
CONFIGURATION = 25
ITERATIONS = 10

model = train_mnist
oracle = Oracle(config)


scheduler = Scheduler(
  model,
  ITERATIONS,
  CONFIGURATION,
  oracle) 

start_time = time.time()
scheduler.initialisation()     
scheduler.loop()     
print("totalt time: " +  str(time.time() - start_time))


iteration: 0
accuracy, 0.107

iteration: 0
accuracy, 0.114

iteration: 0
accuracy, 0.9362

iteration: 0
accuracy, 0.1052

iteration: 0
accuracy, 0.9316

iteration: 0
accuracy, 0.817

iteration: 0
accuracy, 0.9066

iteration: 0
accuracy, 0.9032

iteration: 0
accuracy, 0.0938

iteration: 0
accuracy, 0.9694

iteration: 0
accuracy, 0.121

iteration: 0
accuracy, 0.776

iteration: 0
accuracy, 0.9194

iteration: 0
accuracy, 0.6064

iteration: 0
accuracy, 0.8262

iteration: 0
accuracy, 0.0976

iteration: 0
accuracy, 0.4396

iteration: 0
accuracy, 0.9538

iteration: 0
accuracy, 0.9524

iteration: 0
accuracy, 0.8826

iteration: 0
accuracy, 0.9728

iteration: 0
accuracy, 0.8764

iteration: 0
accuracy, 0.9526

iteration: 0
accuracy, 0.878

iteration: 0
accuracy, 0.837

100%|██████████| 25/25 [02:58<00:00,  7.16s/trial, best loss: -0.9728]

 loss of parent -0.9728

 loss [array(-0.9728)]
not replicated
iteration: 1
accuracy, 0.9764

100%|██████████| 26/26 [00:06<00:00,  3.84trial/s, best loss: -0.9

In [ ]:
for _ in range(1):


  scheduler = Scheduler(
    model,
    ITERATIONS,
    CONFIGURATION,
    oracle) 

  start_time = time.time()
  scheduler.initialisation()     
  scheduler.loop()     
  print("totalt time: " +  str(time.time() - start_time))



Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.

 loss [-0.9778 -0.985  -0.986  -0.99   -0.9914]
not replicated
iteration: 5
accuracy, 0.99

100%|██████████| 1/1 [00:06<00:00,  6.26s/trial, best loss: -0.99]
iteration: 5
accuracy, 0.9904

iteration: 5
accuracy, 0.0976

iteration: 5
accuracy, 0.9582

iteration: 5
accuracy, 0.8852

100%|██████████| 5/5 [00:25<00:00,  5.10s/trial, best loss: -0.9904]

 loss of parent -0.9912

 loss [-0.9778 -0.985  -0.986  -0.99   -0.9912]
not replicated
iteration: 5
accuracy, 0.992

100%|██████████| 1/1 [00:06<00:00,  6.28s/trial, best loss: -0.992]
iteration: 5
accuracy, 0.9844

iteration: 5
accuracy, 0.9794

iteration: 5
accuracy, 0.107

iteration: 5
accuracy, 0.9236

100%|██████████| 5/5 [00:25<00:00,  5.05s/trial, best loss: -0.992]

 loss of parent -0.9906

 loss [-0.9778 -0.9848 -0.9876 -0.9902 -0.9906]
not replicated
iteration: 5
accuracy, 0.9912

100%|██████████| 1/1 [00:06<00:00,  6.38s/trial, best loss: -0.9912]
it

In [ ]:
for _ in range(5):
  oracle = BayesOpt(config)
  start_time = time.time()
  oracle.compute_Once(function)
  print("totalt time: " +  str(time.time() - start_time))

{'b1': 0.9224664001750792, 'b2': 0.9910480360004528, 'droupout_prob': 0.19902167487806088, 'lr': 0.03586505017867996, 'weight_decay': 0.0853378688306543}
iteration: 0
iteration: 1
iteration: 2
iteration: 3
iteration: 4
iteration: 5
iteration: 6
iteration: 7
iteration: 8
iteration: 9
{'b1': 0.9177937724395867, 'b2': 0.9903171175309049, 'droupout_prob': 0.13496184443463988, 'lr': 0.029624673218006593, 'weight_decay': 0.09706943683877561}
iteration: 0
iteration: 1
iteration: 2
iteration: 3
iteration: 4
iteration: 5
iteration: 6
iteration: 7
iteration: 8
iteration: 9
{'b1': 0.9932407205901739, 'b2': 0.9974781387950922, 'droupout_prob': 0.8822746644025263, 'lr': 0.09825412380757259, 'weight_decay': 0.01131624610680608}
iteration: 0
iteration: 1
iteration: 2
iteration: 3
iteration: 4
iteration: 5
iteration: 6
iteration: 7
iteration: 8
iteration: 9
{'b1': 0.9940316365452848, 'b2': 0.9996933287835196, 'droupout_prob': 0.9754202110285426, 'lr': 0.09536494613131066, 'weight_decay': 0.00371370115

[]

In [ ]:

# Oracle (Paul) TODO
path = "/gdrive/MyDrive/"
class Oracle():
    def __init__(self, searchspace ):
        #self.hyperspace is the original (input) searchspace
        self.searchspace = searchspace

    def repeat_good(self,trials, iteration,function,configuration): #add space
        space = copy.deepcopy(configuration)
        for k,v in configuration.items():
            space[k] =  hp.uniform(k,-1e-10+v,v + 1e-10) 

        curr_eval = getattr(trials,'_ids')
        if curr_eval == set():
            curr_eval = 0
        else:
            curr_eval = max(curr_eval) +1
        space["itération"] =  hp.quniform("itération",-.5+iteration,.5+iteration, 1) 
        fmin(function, space, algo=partial(tpe.suggest, n_startup_jobs=1), max_evals=curr_eval
+1, trials=trials)
        
    def compute_once(self,trials, iteration,function): #add space

        space = copy.deepcopy(self.searchspace)
        curr_eval = getattr(trials,'_ids')
        if curr_eval == set():
            curr_eval = 0
        else:
            curr_eval = max(curr_eval) +1
        space["itération"] =  hp.quniform("itération",-.5+iteration,.5+iteration, 1) 
        fmin(function, space, algo=partial(tpe.suggest, n_startup_jobs=1), max_evals=curr_eval
+1, trials=trials)
        
        
    def compute_batch(self,trials, nb_eval, iteration,function): #add space

        space = copy.deepcopy(self.searchspace)
        curr_eval = getattr(trials,'_ids')
        if curr_eval == set():
            curr_eval = 0
        else:
            curr_eval = max(curr_eval) +1
            
        space["itération"] =  hp.quniform("itération",-.5+iteration,.5+iteration, 1) 
        fmin(function, space, algo=partial(tpe.suggest, n_startup_jobs=1), max_evals=curr_eval
+nb_eval, trials=trials)
    


In [ ]:
def test_function(x,models,h,losses, parent_model,k_f,iteration):
    if (isinstance(k_f,list)):
            k=k_f[0]
            Islist = True 
    else:
            k = k_f
            Islist = False
        
    if iteration == 0:
        models[k] = parent_model[k](x)
    else:      
        models[k] = parent_model.adapt(x)
    if(Islist):
        k_f[0] += 1
    
    #for key, value in x.items():
    #        print(key + " "+str(x[key]))

    h[k] = x
    #start_time = time.time()
    models[k].train1()
    loss = models[k].test1()
    test = models[k].val1()
    #print("--- %s seconds ---" % (time.time() - start_time))

    temp = dict(x)
    temp.update({'loss' : loss})
    temp.update({'test' : test})
    fsvnlogger.on_result(temp)

    losses[k] = -loss
    print("accuracy, " + str(loss) + "\n")
    return -loss

In [ ]:
#Normal Random Opt
evals = 25
crypt = np.zeros(evals)
def function(x):
    print(x)
    model = train_mnist(x)
    for _ in range(10):
        model.train1()
        loss = model.test1()
        test = model.val1()
        temp = dict(x)
        temp.update({'loss' : loss})
        temp.update({'test' : test})

        temp.update({'iteration' :  model.i})
        hyperlogger.on_result(temp)
    
  #  loss = model.test1()
    return -loss

In [ ]:
class RandomOpt():
    def __init__(self, searchspace ):
        self.searchspace = searchspace

    def compute_Once(self,function): 
        fmin(function, self.searchspace, algo=partial(tpe.rand.suggest), max_evals=evals, trials=Trials())
    
config= {
     "lr": hp.uniform("lr",0,.1),
     "droupout_prob": hp.uniform("droupout_prob",0,1),
     "weight_decay": hp.uniform("weight_decay",0,.1),
    "b1" : hp.uniform("b1",0.9, 1),
    "b2" : hp.uniform("b2",0.99, 1)
    
}
oracle = RandomOpt(config)

In [ ]:
class RandomLogger(tune.logger.Logger):
    def _init(self):
        progress_file = os.path.join(path, "randomopt_v2.csv")
        self._continuing = os.path.exists(progress_file)
        self._file = open(progress_file, "a")
        self._csv_out = None
    def on_result(self, result):
        tmp = result.copy()
        result = flatten_dict(tmp, delimiter="/")
        if self._csv_out is None:
            self._csv_out = csv.DictWriter(self._file, result.keys())
            self._csv_out.writeheader()

            #if not self._continuing:
        self._csv_out.writerow(
            {k: v
             for k, v in result.items() if k in self._csv_out.fieldnames})
        self._file.flush()
                   
hyperlogger = RandomLogger(config,"")

start_time = time.time()
oracle.compute_Once(function)
print("totalt time: " +  str(time.time() - start_time))


{'b1': 0.9206057713060201, 'b2': 0.9984980969950257, 'droupout_prob': 0.9059718166811926, 'lr': 0.03949001687534575, 'weight_decay': 0.007644962560692282}
iteration: 0
iteration: 1
iteration: 2
iteration: 3
iteration: 4
iteration: 5
iteration: 6
iteration: 7
iteration: 8
iteration: 9
{'b1': 0.9968705555812539, 'b2': 0.9998022546816038, 'droupout_prob': 0.44210063658853516, 'lr': 0.0015720503606858173, 'weight_decay': 0.08720200802352737}
iteration: 0
iteration: 1
iteration: 2
iteration: 3
iteration: 4
iteration: 5
iteration: 6
iteration: 7
iteration: 8
iteration: 9
{'b1': 0.9974049988363484, 'b2': 0.9961005574664765, 'droupout_prob': 0.7966042847912064, 'lr': 0.06725070165730718, 'weight_decay': 0.09384692452986367}
iteration: 0
iteration: 1
iteration: 2
iteration: 3
iteration: 4
iteration: 5
iteration: 6
iteration: 7
iteration: 8
iteration: 9
{'b1': 0.9796626544090346, 'b2': 0.9978668357827766, 'droupout_prob': 0.5392381187152, 'lr': 0.09956569969537885, 'weight_decay': 0.046662080940

In [ ]:
class BayesOpt():
    def __init__(self, searchspace ):
        self.searchspace = searchspace

    def compute_Once(self,function): 
        fmin(function, self.searchspace, algo=partial(tpe.suggest, n_startup_jobs=1), max_evals=evals, trials=Trials())
    
config= {
     "lr": hp.uniform("lr",0,.1),
     "droupout_prob": hp.uniform("droupout_prob",0,1),
     "weight_decay": hp.uniform("weight_decay",0,.1),
    "b1" : hp.uniform("b1",0.9, 1),
    "b2" : hp.uniform("b2",0.99, 1)
    
}
oracle = BayesOpt(config)

In [ ]:
class HyperLogger(tune.logger.Logger):
    def _init(self):
        progress_file = os.path.join(path, "hyperopt_v2.csv")
        self._continuing = os.path.exists(progress_file)
        self._file = open(progress_file, "a")
        self._csv_out = None
    def on_result(self, result):
        tmp = result.copy()
        result = flatten_dict(tmp, delimiter="/")
        if self._csv_out is None:
            self._csv_out = csv.DictWriter(self._file, result.keys())
            self._csv_out.writeheader()

            #if not self._continuing:
        self._csv_out.writerow(
            {k: v
             for k, v in result.items() if k in self._csv_out.fieldnames})
        self._file.flush()
                   
hyperlogger = HyperLogger(config,"")

  


In [ ]:
# A random mnist from the internet to get a correct model to reason about
# IN THIS MODULE: IMPORTS, CNN, TRAIN, TEST, MNIS_FUNCTION, SPACE

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import torch
import torch.optim as optim
import torch.nn as nn
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch.nn.functional as F
from ray import tune
from ray.tune.schedulers import ASHAScheduler
from hyperopt import hp
from ray.tune.suggest.hyperopt import HyperOptSearch
#from ray.tune.suggest.skopt import SkOptSearch
from ray import tune
from ray.tune.suggest.bayesopt import BayesOptSearch
import time
import ray
from ray.tune.schedulers import AsyncHyperBandScheduler
import argparse
from ray.tune.suggest import ConcurrencyLimiter
from ray.tune.schedulers import AsyncHyperBandScheduler
from ray.tune.suggest.nevergrad import NevergradSearch
import json
from ray.tune import Trainable
from ray.tune.schedulers.hb_bohb import HyperBandForBOHB
from ray.tune.suggest import ConcurrencyLimiter
from ray.tune.schedulers import AsyncHyperBandScheduler
from ray.tune.suggest.dragonfly import DragonflySearch
from ray.tune.schedulers import AsyncHyperBandScheduler
import torch
# IN THIS MODULE: IMPORTS, CNN, TRAIN, TEST, MNIS_FUNCTION, SPACE

import numpy as np
import torch
import torch.optim as optim
import torch.nn as nn
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch.nn as nn

import torch.nn.functional as F
from ray import tune
from ray.tune.schedulers import ASHAScheduler 
from ray.tune.schedulers import MedianStoppingRule

from hyperopt import hp
from ray.tune.suggest.hyperopt import HyperOptSearch
#from ray.tune.suggest.skopt import SkOptSearch
from ray import tune
from ray.tune.suggest.bayesopt import BayesOptSearch
import time
import ray
from ray.tune.schedulers import AsyncHyperBandScheduler
import argparse
from ray.tune.suggest import ConcurrencyLimiter
from ray.tune.schedulers import AsyncHyperBandScheduler
from ray.tune.suggest.nevergrad import NevergradSearch
import json
import os
from ray.tune import Trainable
from ray.tune.schedulers.hb_bohb import HyperBandForBOHB
from ray.tune.suggest import ConcurrencyLimiter
from ray.tune.schedulers import AsyncHyperBandScheduler
#from ray.tune.suggest.dragonfly import DragonflySearch
from ray.tune.schedulers import AsyncHyperBandScheduler
import torch
#import adabelief_pytorch
global_checkpoint_period=np.inf
from ray.tune.schedulers.pb2 import PB2

class train_mnist_pb2(tune.Trainable):
    def setup(self, config):
        self.config = {
        "sigmoid_func": 1
      ,  "hidden_dim":43
      ,  "n_layer":2    }
        for key, value in config.items():
            self.config[key] = value
        config = self.config
        
        self.i = 0
        
        #mnist_transforms = transforms.Compose(
        #    [transforms.ToTensor(),
        #     transforms.Normalize((0.1307, ), (0.3081, ))])
        mnist_transforms = transforms.ToTensor()

        self.train_loader = DataLoader(
            datasets.MNIST("/gdrive/MyDrive", train=True, download=True, transform=mnist_transforms),
            batch_size=64,
            shuffle=True)
       # self.test_loader = DataLoader(
       #     datasets.MNIST("~/data", train=False, transform=mnist_transforms),
       #     batch_size=64,
       #     shuffle=True)

        
        test_valid_dataset = datasets.MNIST("/gdrive/MyDrive", train=False, transform=mnist_transforms)
        valid_ratio = 0.5  
        nb_test = int((1.0 - valid_ratio) * len(test_valid_dataset))
        nb_valid =  int(valid_ratio * len(test_valid_dataset))
        test_dataset, val_dataset = torch.utils.data.dataset.random_split(test_valid_dataset, [nb_test, nb_valid])
        self.test_loader =  DataLoader(test_dataset,
            batch_size=64,
            shuffle=True)

        self.val_loader =  DataLoader(val_dataset,
            batch_size=64,
            shuffle=True)

        sigmoid_func_uniq = nn.Tanh()


        self.model = LeNet(192,int(round(config.get("hidden_dim",64))),10,
                    int( round(config.get("n_layer",1))),
                    config.get("droupout_prob",0.5) ,sigmoid_func_uniq)
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.model.to(device)
        if config.get("b1", 0.999)>=1:
          temp = 1 - 1e-10
        else:
          temp = config.get("b1", 0.999)
                
        if config.get("b2", 0.999)>=1:
          temp1 = 1 - 1e-10
        else:
          temp1 = config.get("b2", 0.999)
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr=config.get("lr", 0.01), 
                                     betas=((temp,temp1)),
                                     eps=config.get("eps", 1e-08), 
                                     weight_decay=config.get("weight_decay", 0), 
                                     amsgrad=True)

    

    
    def reset_config(self, config):
        if "lr" in config:
            for param_group in self.optimizer.param_groups:
                param_group["lr"] = config.get("lr", 0.01)
        if "b1" in config:
            for param_group in self.optimizer.param_groups:
                param_group["betas"] = (
                    (1-1e-10 if config.get("b1", 0.999)>=1 else config.get("b1", 0.999),(1-1e-10 if config.get("b2", 0.999)>=1 else config.get("b2", 0.999))))
        if "wd" in config:
            for param_group in self.optimizer.param_groups:
                param_group["weight_decay"] = weight_decay
        
        self.model.adapt(config.get("droupout_prob", 0.5))

        self.config = config
        return True
    
# All NN models should have a function train1 and test1 that calls the common train and test defined above.
# train1 and test1 is then used in the scheduler
    def train1(self):
        print("iteration: " + str(self.i) )
        self.i+=1
        train(self.model, self.optimizer, F.nll_loss, self.train_loader)

    def val1(self):
        return test(self.model, F.nll_loss, self.val_loader)
        
    def test1(self):
        return test(self.model, F.nll_loss, self.test_loader)

    def step(self):
        self.train1()
        return {'loss' : self.test1(), 'test' : self.val1()}
    
    def save_checkpoint(self, checkpoint_dir):
        path = os.path.join(checkpoint_dir, "checkpoint")
        torch.save({
            "model": self.model.state_dict(),
            "optim": self.optimizer.state_dict(),

        }, path)

        return checkpoint_dir

    def load_checkpoint(self, checkpoint_dir):
        path = os.path.join(checkpoint_dir, "checkpoint")
        checkpoint = torch.load(path)
        self.model.load_state_dict(checkpoint["model"])
        self.optimizer.load_state_dict(checkpoint["optim"])
epsilon = 1e-10



algo = HyperOptSearch(metric="loss",
    mode="max")
algo = ConcurrencyLimiter(algo, max_concurrent=25)
scheduler = PB2(
    time_attr="training_iteration",
    perturbation_interval=2,
    hyperparam_bounds={
        # distribution for resampling
     "lr": [0+epsilon, .1] #tune.uniform(1e-4, 0.1 ),#,1e-4), #*10
,     "weight_decay":[0+epsilon, .1]#tune.uniform(1, 5)#,1e-4), #*10 et 0
,     "b1": [.9, 1-epsilon]#,1e-4), #*10 et 0
 ,    "b2": [.99, 1-epsilon] #,1e-4), #*10 et 0
 ,    "droupout_prob": [0+epsilon, 1-epsilon]#,1e-4), #*10 et 0
    }) 

imageSize = 32


class TestLogger(tune.logger.Logger):
    def _init(self):
        progress_file = os.path.join(path, "PB2_2_6.csv")
        self._continuing = os.path.exists(progress_file)
        self._file = open(progress_file, "a")
        self._csv_out = None
    def on_result(self, result):
        tmp = result.copy()
        result = flatten_dict(tmp, delimiter="/")
        if self._csv_out is None:
            self._csv_out = csv.DictWriter(self._file, result.keys())
            self._csv_out.writeheader()
        self._csv_out.writerow(
            {k: v
             for k, v in result.items() if k in self._csv_out.fieldnames})
        self._file.flush()
                   

cst=imageSize
total=cst*cst*3
def trial_name_id(trial):
    return f"{trial.trainable_name}_{trial.trial_id}"
from ray import tune


for _ in range(0):
  ray.shutdown()
  ray.init()
  start_time = time.time()
  analysis = tune.run(
      
  train_mnist_pb2,
  scheduler=scheduler,
  reuse_actors=False,
  search_alg=algo,
  verbose=2,
  checkpoint_at_end=True,
  num_samples=28,
  # export_formats=[ExportFormat.MODEL],
              config= {
  "lr":  tune.uniform(0+epsilon, .1) #tune.uniform(1e-4, 0.1 ),#,1e-4), #*10
  ,     "weight_decay":tune.uniform(0+epsilon, .1)#tune.uniform(1, 5)#,1e-4), #*10 et 0
  ,     "b1": tune.uniform(.9, 1-epsilon)#,1e-4), #*10 et 0
  ,    "b2": tune.uniform(.99, 1-epsilon)#,1e-4), #*10 et 0
  ,    "droupout_prob": tune.uniform(0+epsilon, 1-epsilon)#,1e-4), #*10 et 0
          
      },      stop={
          "training_iteration": 10,
      },        metric="loss",
      mode="max"
,resources_per_trial={'cpu':2 ,'gpu': 0}
              ,     loggers=[TestLogger])
  print("time "+ str(time.time()- start_time))

In [ ]:
from ray.tune.schedulers.hb_bohb import HyperBandForBOHB
from ray.tune.suggest.bohb import *
class TestLogger(tune.logger.Logger):
    def _init(self):
        progress_file = os.path.join(path, "BOHB_Boston1.csv")
        self._continuing = os.path.exists(progress_file)
        self._file = open(progress_file, "a")
        self._csv_out = None
    def on_result(self, result):
        tmp = result.copy()
        result = flatten_dict(tmp, delimiter="/")
        if self._csv_out is None:
            self._csv_out = csv.DictWriter(self._file, result.keys())
            self._csv_out.writeheader()
        self._csv_out.writerow(
            {k: v
             for k, v in result.items() if k in self._csv_out.fieldnames})
        self._file.flush()

config= {
  "lr":  tune.uniform(0+epsilon, .1) #tune.uniform(1e-4, 0.1 ),#,1e-4), #*10
  ,     "weight_decay":tune.uniform(0+epsilon, .1)#tune.uniform(1, 5)#,1e-4), #*10 et 0
  ,     "b1": tune.uniform(.9, 1-epsilon)#,1e-4), #*10 et 0
  ,    "b2": tune.uniform(.99, 1-epsilon)#,1e-4), #*10 et 0
  ,    "droupout_prob": tune.uniform(0+epsilon, 1-epsilon)#,1e-4), #*10 et 0
                  }

for _ in range(10):
    ray.shutdown()
    ray.init()
    start_time = time.time()
    algo = TuneBOHB(metric="loss", mode="max")
    bohb = HyperBandForBOHB(
        time_attr="training_iteration",
        metric="loss",
        mode="max",
        max_t=10)
    analysis = tune.run(train_mnist_pb2, config=config, scheduler=bohb, search_alg=algo,num_samples=25,loggers=[TestLogger],resources_per_trial={'cpu':2 ,'gpu': 1})

    print("time "+ str(time.time()- start_time))

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
!pip install ConfigSpace
!pip install hpbandster


     |████████████████████████████████| 61kB 7.9MB/s 
     |████████████████████████████████| 92kB 9.3MB/s 
  Created wheel for hpbandster: filename=hpbandster-0.7.4-cp37-none-any.whl size=80008 sha256=50808efaa49032be9dfdfc7d3ff24cfedbd8d8aba7f91ac68478dc70e0d6ba03
  Stored in directory: /root/.cache/pip/wheels/9d/57/62/6b00c8011bac96e0c404adc5be4e16964ba4544614240b4e23
  Created wheel for netifaces: filename=netifaces-0.10.9-cp37-cp37m-linux_x86_64.whl size=37425 sha256=164463cc9ed5bcaa1f0ac74459141f827c4cd28c90d60b2cbdf06ccac3f07bae
  Stored in directory: /root/.cache/pip/wheels/23/8f/f3/7054578f04c904f70757c5c85a6e2823baa69d42365526e93d
Successfully built hpbandster netifaces


In [ ]:

import argparse

import ray
from ray import tune
from ray.tune.schedulers.pb2 import PB2
from ray.tune.schedulers import PopulationBasedTraining

from ray.tune.examples.pbt_function import pbt_function


for i in range(1,6):
    class TestLogger(tune.logger.Logger):
        def _init(self):
            progress_file = os.path.join(path, "PBT"+str(i)+".csv")
            self._continuing = os.path.exists(progress_file)
            self._file = open(progress_file, "a")
            self._csv_out = None
        def on_result(self, result):
            tmp = result.copy()
            result = flatten_dict(tmp, delimiter="/")
            if self._csv_out is None:
                self._csv_out = csv.DictWriter(self._file, result.keys())
                self._csv_out.writeheader()
            self._csv_out.writerow(
                {k: v
                 for k, v in result.items() if k in self._csv_out.fieldnames})
            self._file.flush()

    parser = argparse.ArgumentParser()
    parser.add_argument(
        "--smoke-test", action="store_true", help="Finish quickly for testing")
    args, _ = parser.parse_known_args()
    ray.shutdown()
    ray.init()  # force pausing to happen for test

    epsilon = 1e-10
    from ray.tune.suggest.hyperopt import HyperOptSearch
    from ray.tune.suggest import ConcurrencyLimiter

    algo = HyperOptSearch(metric="loss",
        mode="max")
    algo = ConcurrencyLimiter(algo, max_concurrent=4)

    pbt = PopulationBasedTraining(
        perturbation_interval=1,
            time_attr="training_iteration",

        hyperparam_mutations={
            # hyperparameter bounds.
     "lr": [0+epsilon, .1] #tune.uniform(1e-4, 0.1 ),#,1e-4), #*10
,     "weight_decay":[0+epsilon, .1]#tune.uniform(1, 5)#,1e-4), #*10 et 0
,     "b1": [.9, 1-epsilon]#,1e-4), #*10 et 0
 ,    "b2": [.99, 1-epsilon] #,1e-4), #*10 et 0
 ,    "droupout_prob": [0+epsilon, 1-epsilon]#,1e-4), #*10 et 0
        })

    analysis = tune.run(
        train_mnist_pb2,
  checkpoint_at_end=True,
        scheduler=pbt,
        metric="loss",
        mode="max",
        search_alg = algo,
        verbose=0,
        stop={
            "training_iteration": 10,
        },
        num_samples=25,
  reuse_actors=True,
loggers=[TestLogger],
        
          config= {
  "lr":  tune.uniform(0+epsilon, .1) #tune.uniform(1e-4, 0.1 ),#,1e-4), #*10
  ,     "weight_decay":tune.uniform(0+epsilon, .1)#tune.uniform(1, 5)#,1e-4), #*10 et 0
  ,     "b1": tune.uniform(.9, 1-epsilon)#,1e-4), #*10 et 0
  ,    "b2": tune.uniform(.99, 1-epsilon)#,1e-4), #*10 et 0
  ,    "droupout_prob": tune.uniform(0+epsilon, 1-epsilon)#,1e-4), #*10 et 0
                      },resources_per_trial={'cpu':0 ,'gpu': 1})

    print("Best hyperparameters found were: ", analysis.best_config)

2021-03-28 13:07:15,798	INFO services.py:1173 -- View the Ray dashboard at http://127.0.0.1:8265


(pid=10166) iteration: 0
(pid=10167) iteration: 0


2021-03-28 13:07:40,410	INFO pbt.py:482 -- [pbt]: no checkpoint for trial. Skip exploit for Trial train_mnist_pb2_86434558
2021-03-28 13:07:40,537	WARNING trial_runner.py:370 -- Trial Runner checkpointing failed: Checkpoint must not be in-memory.


(pid=10250) iteration: 0


2021-03-28 13:07:51,732	WARNING trial_runner.py:370 -- Trial Runner checkpointing failed: Checkpoint must not be in-memory.


...

(pid=15975) iteration: 9
(pid=15975) iteration: 10


(pid=15975) 2021-03-28 15:22:44,014	INFO trainable.py:329 -- Restored on 172.28.0.2 from checkpoint: /root/ray_results/train_mnist_pb2_2021-03-28_14-54-53/train_mnist_pb2_f0b35e84_23_b1=0.96715,b2=0.99463,droupout_prob=0.37897,lr=0.075946,weight_decay=0.039975_2021-03-28_15-19-07/tmpgbx3og87restore_from_object/./
(pid=15975) 2021-03-28 15:22:44,014	INFO trainable.py:336 -- Current state after restoring: {'_iteration': 9, '_timesteps_total': None, '_time_total': 58.61205697059631, '_episodes_total': None}
2021-03-28 15:22:50,686	WARNING trial_runner.py:370 -- Trial Runner checkpointing failed: Checkpoint must not be in-memory.
(pid=15975) 2021-03-28 15:22:50,717	INFO trainable.py:329 -- Restored on 172.28.0.2 from checkpoint: /root/ray_results/train_mnist_pb2_2021-03-28_14-54-53/train_mnist_pb2_1b27530a_24_b1=0.9829,b2=0.99682,droupout_prob=0.29697,lr=0.08724,weight_decay=0.062504_2021-03-28_15-20-19/tmpkxgoqljzrestore_from_object/./
(pid=15975) 2021-03-28 15:22:50,717	INFO trainable.py

(pid=15975) iteration: 11
(pid=15975) iteration: 12


(pid=15975) 2021-03-28 15:22:57,434	INFO trainable.py:329 -- Restored on 172.28.0.2 from checkpoint: /root/ray_results/train_mnist_pb2_2021-03-28_14-54-53/train_mnist_pb2_45e1daa2_25_b1=0.90703,b2=0.99618,droupout_prob=0.022314,lr=0.071447,weight_decay=0.04881_2021-03-28_15-21-30/tmpv1thg7b8restore_from_object/./
(pid=15975) 2021-03-28 15:22:57,434	INFO trainable.py:336 -- Current state after restoring: {'_iteration': 9, '_timesteps_total': None, '_time_total': 60.20059013366699, '_episodes_total': None}
2021-03-28 15:23:04,106	WARNING trial_runner.py:370 -- Trial Runner checkpointing failed: Checkpoint must not be in-memory.


Best hyperparameters found were:  {'lr': 0.1, 'weight_decay': 0.1, 'b1': 0.9, 'b2': 0.9999999999, 'droupout_prob': 1e-10}


In [ ]:
from ray.tune.schedulers.pb2_utils import normalize, optimize_acq, \
            select_length, UCB, standardize, TV_SquaredExp